In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

BATCH_SIZE = 10
MIN_number_of_attributes = 2
MAX_number_of_attributes = 35

number_of_NN_repetitions = 10
#number_of_attributes = 20

print(tf.__version__)

2.7.0


In [ ]:
path_training ="/content/drive/MyDrive/csv-files/01_Atributos_Weve.csv"
path_test ="/content/drive/MyDrive/csv-files/02_Atributos_Weve.csv"
df_raw = pd.read_csv(path_training)
df_test_raw = pd.read_csv(path_test)
df_raw
target = df_raw.pop('target')
server_port = df_raw.pop('ServerPort')
client_port = df_raw.pop('ClientPort')

target_test = df_test_raw.pop('target')
server_port_test = df_test_raw.pop('ServerPort')
client_port = df_test_raw.pop('ClientPort')


#copy HERE the sorted attributed list generated with IC-NN-Permutations.ipynb 
attributes_array = ['83', '82', '79', '22', '90', '84', '88', '15', '60', '17', '29', '80', '89', '87', '81', '59', '45', '47', '10', '31', '37', '33', '36', '64', '62', '38', '32', '24', '34', '91', '63', '92', '61', '48', '46']

#attributes_array = ['10','15','17','22','24','29','31','32','33','34','36','37','38','45','46','47','48','59','60','61','62','63','64','75','76','77','78','79','80','81','82','83','84','87','88','89','90','91','92']                    

In [ ]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target_test = target_test.apply(lambda x: classes.index(x))

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
for number_of_attributes in range(MIN_number_of_attributes ,MAX_number_of_attributes+1):
  # Select a sub-dataframe from the main dataframe with a lower number of attributes
  df = df_raw[attributes_array[0:number_of_attributes]]
  df_test = df_test_raw[attributes_array[0:number_of_attributes]]
  #print(df_test)

  # Combine the data with possible classes
  numeric_feature_names = attributes_array[0:number_of_attributes]
  numeric_features = df[numeric_feature_names] # Data for training
  numeric_features_test = df_test[numeric_feature_names] # Data for testing

  # Converts pandas dataframe to tensorflow object
  numeric_features = tf.convert_to_tensor(numeric_features)
  #numeric_features_test = tf.convert_to_tensor(numeric_features_test) botei na célula abaixo

  # Normalize the data
  normalizer = tf.keras.layers.Normalization(axis=-1)
  normalizer.adapt(numeric_features)
  #normalizer.adapt(numeric_features_test)

  # Execute Trainning and Testing
  #model = get_basic_model()
  print('Starting training for',number_of_attributes,'attributes')
  train_accuracy = []
  test_acc = []
  for i in range(number_of_NN_repetitions):
    model = get_basic_model()
    history = model.fit(numeric_features, target, epochs=10, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
    one_train_accuracy = history.history['accuracy']

    test_loss, one_test_acc =  model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE)
    train_accuracy.append(one_train_accuracy[-1])
    test_acc.append(one_test_acc)
    print(f"{i+1}: train_accuracy:{train_accuracy} test_acc:{test_acc}")

  train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
  test_acc_mean = sum(test_acc) / len(test_acc)
  print(f'train_accuracy_mean:{train_accuracy_mean} test_acc_mean: {test_acc_mean}')
  # Date for report file
  current_date_and_time = datetime.date.today()
  current_date_and_time_string = str(current_date_and_time)
  # Report file content
  report = {
      "datetime": current_date_and_time_string,
      "training_dataset_shape":  df.shape,
      "test_dataset_shape": df_test.shape,
      "attributes": attributes_array[0:df.shape[1]],
      "number_of_attributes": number_of_attributes,
      "accuracy_train": train_accuracy_mean,
      "accuracy_test": test_acc_mean
  }
  # Writes into the file
  title = "/content/drive/MyDrive/IC-Skynet/NNreports/%s"%current_date_and_time_string + " With %d Attributes.json"%number_of_attributes
  with open(title, "w") as f:
    json.dump(report, f)
  with open(title, "r") as f:
    rep = json.load(f)
  #display(rep)

  #print(rep["datetime"])



Starting training for 34 attributes
2381/2381 - 2s - loss: 0.2067 - accuracy: 0.9504 - 2s/epoch - 927us/step
1: train_accuracy:[0.975787341594696] test_acc:[0.950380265712738]
train_accuracy_mean:0.975787341594696 test_acc_mean: 0.950380265712738
Starting training for 35 attributes
2381/2381 - 2s - loss: 0.1634 - accuracy: 0.9570 - 2s/epoch - 944us/step
1: train_accuracy:[0.9783211946487427] test_acc:[0.956976592540741]
train_accuracy_mean:0.9783211946487427 test_acc_mean: 0.956976592540741


In [ ]:
# Feature Importance using connection weights

# #layer0_weights = model.layers[0].get_weights()[0] #weight
# #layer0_biases  = model.layers[0].get_weights()[1] #bias
 #weight
# #layer1_biases  = model.layers[1].get_weights()[1] #bias
# #print('Layer 1 weights:',layer1_weights)
# # print('\n')

layer1_weights = model.layers[1].get_weights()[0]

weight_attribute = []
for attr_i, weights in enumerate(layer1_weights):
  weight_attribute.append((weights.sum(), numeric_feature_names[attr_i]))

weight_attribute = sorted(weight_attribute)
print(weight_attribute)

attributes = [x for _, x in weight_attribute]
print(attributes)

[(-6.9801636, '59'), (-5.5289674, '60'), (-4.315676, '83'), (-2.9429455, '34'), (-2.9264152, '79'), (-2.4212039, '46'), (-2.272394, '45'), (-2.1140418, '82'), (-1.9522867, '31'), (-1.8983507, '47'), (-1.7361214, '32'), (-1.722243, '48'), (-1.4368861, '24'), (-1.3417174, '15'), (-1.2784703, '33'), (-1.2014674, '92'), (-1.0711381, '84'), (-0.9850577, '61'), (-0.97068053, '36'), (-0.52946246, '22'), (-0.22432101, '63'), (-0.1378617, '91'), (-0.036468223, '10'), (0.055996045, '89'), (0.1575387, '29'), (0.364222, '38'), (0.71299183, '80'), (0.8211733, '17'), (0.82525015, '64'), (0.909147, '37'), (1.0775454, '62'), (1.1865104, '87'), (1.2435303, '81'), (1.5492694, '90'), (2.3258693, '88')]
['59', '60', '83', '34', '79', '46', '45', '82', '31', '47', '32', '48', '24', '15', '33', '92', '84', '61', '36', '22', '63', '91', '10', '89', '29', '38', '80', '17', '64', '37', '62', '87', '81', '90', '88']
